In [1]:
!pip install beautifulsoup4 pandas selenium webdriver-manager

Defaulting to user installation because normal site-packages is not writeable


**Setup-Teste inicial para o web scrapping**

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = 'https://www.moneytimes.com.br/ultimas-noticias/'

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

response = requests.get(url, headers=headers)

print(f"Status Code: {response.status_code}")

Status Code: 200


**Scrapping para o site moneytimes.com.br**

In [3]:
i = 0
dados = []
while response.status_code == 200 and i<=10:

  soup = BeautifulSoup(response.content, 'html.parser')

  divNoticia = soup.find_all(class_='news-item')

  print(f"Encontrados {len(divNoticia)} elementos com a classe 'news-item' na página {i}")

  for k, item in enumerate(divNoticia):
        try:

            titulo = item.find('h2', class_= 'news-item__title')
            data = item.find('span', class_='date')
            if titulo:
                titulo = titulo.get_text(strip=True)
                print(f"{i+1}. {titulo}")
                if data:
                    data = data.get_text(strip=True)
                    registro = {
                    'Titulo': titulo,
                    'Data': data
                }
            else:
                print(f"{k+1}. [Titulo não encontrado dentro do bloco]")
            dados.append(registro)
        except Exception as e:
            print(f"Erro ao ler item {k}: {e}")
  i+=1
  url = f'https://www.moneytimes.com.br/empresas/page/{i}/'
  headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
  response = requests.get(url, headers=headers)

scrapping = pd.DataFrame(dados)

scrapping.tail()


Encontrados 21 elementos com a classe 'news-item' na página 0
1. MBRF (MBRF3) puxa queda do Ibovespa na semana, apesar de duas notícias positivas; entenda
1. Cogna (COGN3) lidera os ganhos do Ibovespa, enquanto MBRF (MBRF3) é a ação com pior desempenho; veja os destaques da semana
1. Medida do governo pode colocar até um ‘14º salário’ no bolso de 25% dos investidores brasileiros: saiba como transformá-lo em renda vitalícia
1. A vaca mais cara do mundo: ‘Donna’ é vendida por R$ 54 milhões
1. A bordo da noite: como é a viagem no trem noturno da Vale?
1. Jair Bolsonaro é preso pela Polícia Federal
1. Ibovespa a 174 mil pontos em 2026? EQI aposta que a bolsa ainda tem espaço para subir mais
1. Neoenergia (NEOE3) vende hidrelétrica em MT por R$ 2,5 bilhões e mantém participação indireta de 25%
1. Ibovespa destoa de Wall Street e encerra semana em queda; dólar sobe a R$ 5,40
1. Wall Street tem forte recuperação e sobe 1% de olho em novo corte nos juros
1. Eleições no Chile selam a guinada co

,Titulo,Data
106,XP (XPBR31) anuncia R$ 500 milhões em dividend...,4 dia(s) atrás
107,MBRF (MBRF3) conclui processo para usar apenas...,4 dia(s) atrás
108,Petrobras (PETR4): Nova descoberta de petróleo...,4 dia(s) atrás
109,Banco do Brasil (BBAS3) na briga pelos milioná...,4 dia(s) atrás
110,Banco do Brasil (BBAS3) na briga pelos milioná...,4 dia(s) atrás


In [4]:
scrapping.to_csv('datasetSentimento.csv', index=False, encoding='utf-8-sig')